In [68]:
import time
import hashlib
import json
import random
from pprint import pprint
from datetime import datetime as dt
from dataclasses import dataclass
from collections import namedtuple


payloads = [
    "ty jesteś gwiazda",
    "z tobą jest jazda",
    "chciałbym cię porwać do swego gniazda",
    "dla ciebie kawior, szampan , truskawki",
    "i jakie tylko zechcesz zabawki"]









# Fundament blockchain - Block

In [6]:

@dataclass
class Block:
    index: int
    payload: str
    prev_hash: str    
    timestamp = time.time()    

    def __repr__(self):
        return f'[Block #{self.index}, payload={self.payload}, ' \
               f'hash={self.hash()}, datetime={dt.fromtimestamp(self.timestamp)}]'

    def _sha(self):
        sha = hashlib.sha256()
        sha.update(bytes(self.index))
        sha.update((str(self.timestamp) + json.dumps(self.payload) +
                    self.prev_hash).encode('utf-8'))
        return sha

    def hash(self):
        return self._sha().hexdigest()


# Blockchain

In [7]:
class Blockchain(object):
    def __init__(self):
        self.index = 0
        self.blocks = []
        self.last_hash = ''

    def __repr__(self):
        return f'[Blockchain, height={len(self.blocks)}, ' \
               f'last 5 blocks={str(self.blocks[-5:])}]'

    def add_block(self, payload):
        block = Block(self.index, payload, self.last_hash)
        self.blocks.append(block)
        self.index += 1
        self.last_hash = block.hash()
        return block
    
    

In [43]:
blockchain = Blockchain()
blockchain.add_block('w ogóle centralnie kamieniem go bez kitu')


[Block #0, payload=w ogóle centralnie kamieniem go bez kitu, hash=61207218a0238107ca2e5fc8f8b359587fa9a66218fc0babd73981fc9fdff313, datetime=2018-09-18 11:51:47.376633]

# Dodajmy kilka bloków

In [44]:
for i in range(10):
    payload = f"[{i}] {random.choice(payloads)}"
    block = blockchain.add_block(payload)
    print(f"Block added:\n \t {block}")

Block added:
 	 [Block #1, payload=[0] z tobą jest jazda, hash=b716ad20f5251a932e4778bd7f2c183ab6d1aec2d262b7b4843083685fc0a77f, datetime=2018-09-18 11:51:47.376633]
Block added:
 	 [Block #2, payload=[1] dla ciebie kawior, szampan , truskawki, hash=e30200b8aaa8752c3628f41078304a8e0b513cdbffe5b18efa76523fa2025fa6, datetime=2018-09-18 11:51:47.376633]
Block added:
 	 [Block #3, payload=[2] i jakie tylko zechcesz zabawki, hash=1205268c18c2e2ba5d5645118d44bee972dc37024166f40021f5e7e98f641924, datetime=2018-09-18 11:51:47.376633]
Block added:
 	 [Block #4, payload=[3] chciałbym cię porwać do swego gniazda, hash=9cbaf332eab9e57e8823ae779de50b5fd89c52ae400152f765f5ef1d91a64aa6, datetime=2018-09-18 11:51:47.376633]
Block added:
 	 [Block #5, payload=[4] i jakie tylko zechcesz zabawki, hash=1e7204caf3a65649b9f00e49ee96131de51a17976a34e2cb3db246b859fa6cc5, datetime=2018-09-18 11:51:47.376633]
Block added:
 	 [Block #6, payload=[5] i jakie tylko zechcesz zabawki, hash=80abeda656fb75e7b06b6328f0a

# Weryfikacja spójności danych

In [45]:
def verify(blockchain):
    for block, prev_block in zip(blockchain.blocks[1:], blockchain.blocks):
        if block.prev_hash != prev_block.hash():
            print(f'!!! Blockchain verification FAILED on block #{block.index} - {block.payload} !!!')
            break
    else:
        print('Blockchain verification PASSED')

## Weryfikacja poprawnych danych

In [46]:
verify(blockchain)

Blockchain verification PASSED


## Weryfikacja po drobnej zmianie w bloku

In [47]:
blockchain.blocks[3].payload += " Jam jest 444"
print(blockchain.blocks[3].payload)


[2] i jakie tylko zechcesz zabawki Jam jest 444


In [86]:
%%time
verify(blockchain)

!!! Blockchain verification FAILED on block #4 - [3] chciałbym cię porwać do swego gniazda !!!
CPU times: user 421 µs, sys: 0 ns, total: 421 µs
Wall time: 334 µs


# Teraz na poważnie - transakcje


## Definicja transakcji

In [59]:
Transaction = namedtuple('Transaction', 'addr_from, addr_to, amount')

In [60]:
def create_test_transactions(blockchain):
    blockchain.add_block([Transaction('1FDHiLF1CLnFZ2BZtPakPTY2sk2M8AJkiA',
                                  '19PcPjKGiwdcTwMibqJVRAPY9sGx4XcRAx',
                                  1.0)])

    blockchain.add_block([Transaction('12vi3T1DifT4y7tyz5TGbyFQwj2opmmaia',
                                      '1Gdw1B4dMTZHQUzv7ggnPmZWap9pjzyn4d',
                                      2.3), 
                          Transaction('1PTTZ2gKWBKPuC8JKtFd5sDbT4XQgEzvSL',
                                      '12FuBeX5Ruyd4UxeCeHuAABuQ9cD2aGQKo',
                                      10.0)])

    blockchain.add_block([Transaction('19PcPjKGiwdcTwMibqJVRAPY9sGx4XcRAx',
                                      '1FDHiLF1CLnFZ2BZtPakPTY2sk2M8AJkiA',
                                      1.0)])


In [63]:
serious_blockchain = Blockchain()
serious_blockchain.add_block("w ogóle centralnie kamieniem go bez kitu")

[Block #0, payload=w ogóle centralnie kamieniem go bez kitu, hash=61207218a0238107ca2e5fc8f8b359587fa9a66218fc0babd73981fc9fdff313, datetime=2018-09-18 11:51:47.376633]

In [84]:
%%time
create_test_transactions(serious_blockchain)

CPU times: user 267 µs, sys: 0 ns, total: 267 µs
Wall time: 277 µs


In [73]:
serious_blockchain.blocks

[[Block #0, payload=w ogóle centralnie kamieniem go bez kitu, hash=61207218a0238107ca2e5fc8f8b359587fa9a66218fc0babd73981fc9fdff313, datetime=2018-09-18 11:51:47.376633],
 [Block #1, payload=[Transaction(addr_from='1FDHiLF1CLnFZ2BZtPakPTY2sk2M8AJkiA', addr_to='19PcPjKGiwdcTwMibqJVRAPY9sGx4XcRAx', amount=1.0)], hash=97c30e267644b8c8d8c12d619063cc51ab26e02026859c16f79125b4b85f0ac9, datetime=2018-09-18 11:51:47.376633],
 [Block #2, payload=[Transaction(addr_from='12vi3T1DifT4y7tyz5TGbyFQwj2opmmaia', addr_to='1Gdw1B4dMTZHQUzv7ggnPmZWap9pjzyn4d', amount=2.3), Transaction(addr_from='1PTTZ2gKWBKPuC8JKtFd5sDbT4XQgEzvSL', addr_to='12FuBeX5Ruyd4UxeCeHuAABuQ9cD2aGQKo', amount=10.0)], hash=f0db75dddd1e6eb08d8504ad1ed19eae01ce8267c87fc05b91eee15af0a4cd53, datetime=2018-09-18 11:51:47.376633],
 [Block #3, payload=[Transaction(addr_from='19PcPjKGiwdcTwMibqJVRAPY9sGx4XcRAx', addr_to='1FDHiLF1CLnFZ2BZtPakPTY2sk2M8AJkiA', amount=1.0)], hash=b5fafe06ac90cc6e8ed373b09f3c6d695b8130d12bf7449512d08c613bd7b29

In [74]:
verify(serious_blockchain)

Blockchain verification PASSED


# Mining

In [142]:
class Block:
    def __init__(self, index, payload, prev_hash):
        self.index = index
        self.timestamp = time.time()
        self.payload = payload
        self.prev_hash = prev_hash
        self.nonce = 0
        self.difficulty = 0

    def __repr__(self):
        return f'[Block #{self.index}, payload={self.payload}, ' \
               f'hash={self.hash()}, nonce={self.nonce}, diff={self.difficulty}, ' \
               f'datetime={dt.fromtimestamp(self.timestamp)}, ' \
               f'prev_hash={self.prev_hash}]'

    def _sha(self):
        sha = hashlib.sha256()
        sha.update(bytes(self.index) + bytes(self.nonce) + bytes(self.difficulty))
        sha.update((str(self.timestamp) + json.dumps(self.payload) + 
                    self.prev_hash).encode('utf-8'))
        return sha

    def hash(self):
        return self._sha().hexdigest()

    def difficulty_matches(self):
        print("sha:", self._sha().digest())
        bits = ''.join(f"{c:08b}" for c in self._sha().digest())
        print("Bits", bits)
        prefix = bits[:self.difficulty]
        print("prefix", prefix)
        mask = '0' * self.difficulty
        print(f"mask: {mask}")
        return prefix == mask

    def mine(self, difficulty):
        self.difficulty = difficulty
        while not self.difficulty_matches():
            print("nonce", self.nonce)
            self.nonce += 1


class Blockchain(object):
    def __init__(self):
        self.index = 0
        self.last_hash = ''
        self.blocks = []
        self.difficulty = 0
        self.add_block('The Times 03/Jan/2009 Chancellor on brink of second bailout ' \
                       'for banks')

    def __repr__(self):
        return f'[Blockchain, height={len(self.blocks)}, ' \
               f'last 5 blocks={str(self.blocks[-5:])}]'

    def add_block(self, payload):
        block = Block(self.index, payload, self.last_hash)
        block.mine(self.difficulty)
        self.blocks.append(block)
        self.index += 1
        self.last_hash = block.hash()
        return block
    
    def verify(self):
        for block, prev_block in zip(self.blocks[1:], self.blocks):
            if block.prev_hash != prev_block.hash():
                print(f'Verification FAILED on block #{block.index}')
                break
        else:
            print('Verification PASSED')

In [143]:
hard_blockchain = Blockchain()

sha: b'\xc2\xb5u\xe9g\x07\xd3\xd9{\xac\tS\x97\n".<\x08\xe6He\xc1\x11\xd2m\xd6\xf37\x9e\xb3IW'
Bits 1100001010110101011101011110100101100111000001111101001111011001011110111010110000001001010100111001011100001010001000100010111000111100000010001110011001001000011001011100000100010001110100100110110111010110111100110011011110011110101100110100100101010111
prefix 
mask: 


In [144]:
%%time
create_test_transactions(hard_blockchain)

sha: b'\x0el;gwI\xc3t\xb5\xb3\x10\xc2\xc6\x8f9Jb\x97\x94\xe3\x00.\x04\x1a\xcd\x1d\xb3\xbbS*\xc6`'
Bits 0000111001101100001110110110011101110111010010011100001101110100101101011011001100010000110000101100011010001111001110010100101001100010100101111001010011100011000000000010111000000100000110101100110100011101101100111011101101010011001010101100011001100000
prefix 
mask: 
sha: b"/\xfdC\x8a-\xd7\x96\xe0\x95\n\x99\x9c\x1c\xae\xcd\xca\x1efWg\x82\xe9aw\x96\x97\xad\x99\xf5\xd4B'"
Bits 0010111111111101010000111000101000101101110101111001011011100000100101010000101010011001100111000001110010101110110011011100101000011110011001100101011101100111100000101110100101100001011101111001011010010111101011011001100111110101110101000100001000100111
prefix 
mask: 
sha: b'I\xe7\xa3\xbb\x18\xa8\xeb\xf5\x92\x1b\xde<\xfb\xe7\xe9l\xfd\x1a\xa4\xb1\x08\xc4J\xfc\x9c\xd1\xf1PiW\xe7/'
Bits 01001001111001111010001110111011000110001010100011101011111101011001001000011011110111100011110011111011111001111110100101101

In [145]:
hard_blockchain.verify()

Verification PASSED


## Zwiększamy poziom trudności...

In [146]:
%%time
much_harder_blockchain = Blockchain()
much_harder_blockchain.difficulty = 5  # hardcore level
create_test_transactions(much_harder_blockchain)

sha: b'\xd2e\xd3\x8f(\xeb\xd6\x98\tK\xb7\x99\xb4\x8c(\xbe|\x91i\x13W\xc4\x11\xb8\x0ck\x16\x03\x82\xe0]\xcd'
Bits 1101001001100101110100111000111100101000111010111101011010011000000010010100101110110111100110011011010010001100001010001011111001111100100100010110100100010011010101111100010000010001101110000000110001101011000101100000001110000010111000000101110111001101
prefix 
mask: 
sha: b'\x86\xf0\xb4\\\x1f\x8b\xa6\xda\x02w\r}\xd6\xc1\x17d\x0c\x9c\x8e\xe1\xd2\xbf(\xe3\xf0\x07f\xb4f\x96;\xbf'
Bits 1000011011110000101101000101110000011111100010111010011011011010000000100111011100001101011111011101011011000001000101110110010000001100100111001000111011100001110100101011111100101000111000111111000000000111011001101011010001100110100101100011101110111111
prefix 10000
mask: 00000
nonce 0
sha: b"\xc2\xee|I\xe12\xaf)\x89\x84\xc6\x15\r\xaa.\xac\xef]\x8e\xbd\xfe5\x05\x82_\xb5\x964'\x0fj\xac"
Bits 11000010111011100111110001001001111000010011001010101111001010011000100110000100110001100001010100001

In [100]:
verify(harder_blockchain)

Blockchain verification PASSED
